# Experimentos de Métodos de Optimización

Notebook generado automáticamente a partir del script para comparar:
- Método de máximo descenso con Armijo
- Método de región de confianza con punto de Cauchy

Ejemplo de uso en una celda de código:
```python
import experiments_mo_jupyter as mo
df = mo.run_all()
df
```


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Utilidad del script:
- Objetivo: f(x,y) = (exp(x^2) + y^4) / (y^2 + 1)
- Métodos: Máximo descenso (Armijo) y Región de confianza (punto de Cauchy)
- Salidas: figuras y tabla para LaTeX
"""
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
from math import isfinite

# ======================
# Definiciones simbólicas
# ======================
x, y = sp.symbols('x y', real=True)
f_expr = (sp.exp(x**2) + y**4) / (y**2 + 1)
grad_expr = [sp.diff(f_expr, v) for v in (x, y)]
hess_expr = sp.hessian(f_expr, (x, y))

f = sp.lambdify((x, y), f_expr, 'numpy')
grad = sp.lambdify((x, y), grad_expr, 'numpy')
hess = sp.lambdify((x, y), hess_expr, 'numpy')

# ---------- Armijo ----------
def armijo(f, grad, xk, dk, alpha0=1.0, c=1e-4, tau=0.5, max_iter=30):
    fx = f(*xk)
    gk = np.array(grad(*xk), dtype=float).reshape(-1)
    dphi0 = gk @ dk
    alpha = alpha0
    for _ in range(max_iter):
        x_new = xk + alpha * dk
        fx_new = f(*x_new)
        if np.isfinite(fx_new) and fx_new <= fx + c * alpha * dphi0:
            return alpha
        alpha *= tau
    return alpha

# ---------- Steepest Descent ----------
def steepest_descent(x0, max_iter=200, tol=1e-8, alpha0=1.0):
    xk = np.array(x0, dtype=float)
    traj = [xk.copy()]
    hist = []

    for k in range(max_iter):
        gk = np.array(grad(*xk), dtype=float).reshape(-1)
        fx = f(*xk)
        ng = np.linalg.norm(gk)

        hist.append(dict(k=k, fx=float(fx), ng=float(ng),
                         x=float(xk[0]), y=float(xk[1])))

        if (not isfinite(fx) or not np.isfinite(gk).all()
                or ng < tol):
            break

        dk = -gk
        alpha = armijo(f, grad, xk, dk, alpha0=alpha0)
        xk = xk + alpha * dk
        traj.append(xk.copy())

    return np.array(traj), pd.DataFrame(hist)

# ---------- Trust Region (Cauchy) ----------
def trust_region(x0, max_iter=200, tol=1e-8, Delta0=1.0, eta=0.1):
    xk = np.array(x0, dtype=float)
    Delta = Delta0
    traj = [xk.copy()]
    hist = []

    for k in range(max_iter):
        gk = np.array(grad(*xk), dtype=float).reshape(-1)
        Bk = np.array(hess(*xk), dtype=float)
        fx = f(*xk)
        ng = np.linalg.norm(gk)

        hist.append(dict(k=k, fx=float(fx), ng=float(ng),
                         Delta=float(Delta),
                         x=float(xk[0]), y=float(xk[1])))

        if (not isfinite(fx) or not np.isfinite(gk).all()
                or not np.isfinite(Bk).all() or ng < tol):
            break

        gBg = float(gk @ (Bk @ gk))
        if gBg <= 0:
            tau = 1.0
        else:
            tau = min(ng**3 / (Delta * gBg), 1.0)

        pk = -(tau * Delta / max(ng, 1e-14)) * gk

        mk0 = fx
        mkp = fx + gk @ pk + 0.5 * pk @ (Bk @ pk)
        pred_red = mk0 - mkp

        xtrial = xk + pk
        fx_trial = f(*xtrial)
        if not np.isfinite(fx_trial) or not isfinite(pred_red) or pred_red == 0:
            # si el modelo se vuelve loco, reducimos región y seguimos
            Delta *= 0.25
            continue

        ared = fx - fx_trial
        rho = ared / pred_red

        if rho < 0.25:
            Delta *= 0.25
        else:
            if rho > 0.75 and abs(np.linalg.norm(pk) - Delta) < 1e-12:
                Delta = min(2 * Delta, 100.0)

        if rho > eta:
            xk = xtrial
            traj.append(xk.copy())

    return np.array(traj), pd.DataFrame(hist)

# ---------- Plots auxiliares robustos ----------
def _safe_f_grid(xlim, ylim, n=300):
    X = np.linspace(*xlim, n)
    Y = np.linspace(*ylim, n)
    XX, YY = np.meshgrid(X, Y)
    ZZ = f(XX, YY)
    ZZ = np.where(np.isfinite(ZZ), ZZ, np.nan)
    finite_vals = ZZ[np.isfinite(ZZ)]
    if finite_vals.size == 0:
        return XX, YY, ZZ
    zmin = np.nanmin(finite_vals)
    zmax = np.nanpercentile(finite_vals, 99)  # recortar outliers
    ZZ = np.clip(ZZ, zmin, zmax)
    return XX, YY, ZZ

def plot_surface_and_contours(xlim=(-3,3), ylim=(-3,3), prefix="fig"):
    XX, YY, ZZ = _safe_f_grid(xlim, ylim, n=300)

    # Contornos
    plt.figure(figsize=(7,6))
    cs = plt.contour(XX, YY, ZZ, levels=30)
    if cs.levels.size > 0:
        plt.clabel(cs, inline=1, fontsize=8)
    plt.xlabel("x"); plt.ylabel("y"); plt.title("Contornos de f(x,y)")
    plt.tight_layout(); plt.savefig(prefix + "_contours.png", dpi=160); plt.close()

    # Superficie 3D
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    fig = plt.figure(figsize=(7,6))
    ax = fig.add_subplot(111, projection='3d')
    XXs, YYs, ZZs = _safe_f_grid(xlim, ylim, n=120)
    ax.plot_surface(XXs, YYs, ZZs, linewidth=0, antialiased=False)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("f")
    ax.set_title("Superficie de f(x,y)")
    plt.tight_layout(); plt.savefig(prefix + "_surface.png", dpi=160); plt.close()

def plot_trajectories_over_contour(trajs, title, fname):
    # Calcular rango dinámico según las trayectorias
    all_x = np.concatenate([T[:,0] for T in trajs.values() if len(T)])
    all_y = np.concatenate([T[:,1] for T in trajs.values() if len(T)])
    margin = 0.3
    x_min, x_max = all_x.min() - margin, all_x.max() + margin
    y_min, y_max = all_y.min() - margin, all_y.max() + margin

    # Limitar a una región razonable para evitar overflow extremo
    xlim = (max(-4.0, x_min), min(4.0, x_max))
    ylim = (max(-4.0, y_min), min(4.0, y_max))

    XX, YY, ZZ = _safe_f_grid(xlim, ylim, n=300)

    plt.figure(figsize=(7,6))
    cs = plt.contour(XX, YY, ZZ, levels=30)
    if cs.levels.size > 0:
        plt.clabel(cs, inline=1, fontsize=7)

    for x0, T in trajs.items():
        if len(T) == 0:
            continue
        plt.plot(T[:,0], T[:,1], marker='o', markersize=2,
                 linewidth=1.0, label=f"init {x0}")
        plt.scatter([T[0,0]],[T[0,1]], marker='x')    # inicio
        plt.scatter([T[-1,0]],[T[-1,1]], marker='*')  # final

    plt.xlim(xlim); plt.ylim(ylim)
    plt.legend(fontsize=7, loc="best")
    plt.xlabel("x"); plt.ylabel("y"); plt.title(title)
    plt.tight_layout(); plt.savefig(fname, dpi=160); plt.close()

def plot_convergence(hist_map, method_name, fname,
                     max_iters=60, log_scale=True):
    """
    Grafica la convergencia del valor de la función objetivo f(x_k)
    para cada punto inicial.

    - max_iters: número máximo de iteraciones a mostrar (None = todas)
    - log_scale: si True, grafica log10(f(x_k)) para comprimir la escala.
    """
    plt.figure(figsize=(7,5))

    for x0, hist in hist_map.items():
        if len(hist) == 0:
            continue

        # Recortar número de iteraciones mostradas (solo para la gráfica)
        if max_iters is not None and len(hist) > max_iters:
            hplot = hist.iloc[:max_iters]
        else:
            hplot = hist

        k_vals = hplot['k'].to_numpy()
        f_vals = hplot['fx'].to_numpy()

        # Filtrar valores no finitos
        mask = np.isfinite(f_vals)
        k_vals = k_vals[mask]
        f_vals = f_vals[mask]

        if len(f_vals) == 0:
            continue

        if log_scale:
            # f(x,y) > 0 siempre, pero por seguridad clip
            f_vals = np.clip(f_vals, 1e-12, None)
            y_vals = np.log10(f_vals)
            ylabel = r"$\log_{10} f(x_k)$"
        else:
            y_vals = f_vals
            ylabel = r"$f(x_k)$"

        plt.plot(k_vals, y_vals, linewidth=1.0,
                 label=f"init {x0}")

    plt.xlabel("k")
    plt.ylabel(ylabel)
    plt.title(f"Convergencia de f - {method_name}")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend(fontsize=7, loc="best")
    plt.tight_layout()
    plt.savefig(fname, dpi=160)
    plt.close()


# ---------- Tabla LaTeX ----------
def save_table_tex(df, tex_path):
    tbl = df.copy()
    for col in ['SD_fx_final','TR_fx_final',
                'SD_grad_norm_final','TR_grad_norm_final']:
        if col in tbl.columns:
            tbl[col] = tbl[col].map(lambda v: f"{v:.6g}" if pd.notnull(v) else "--")

    tbl['SD_x_final'] = tbl['SD_x_final'].map(
        lambda v: f"({v[0]:.4g},{v[1]:.4g})" if isinstance(v, (list, tuple)) else "--"
    )
    tbl['TR_x_final'] = tbl['TR_x_final'].map(
        lambda v: f"({v[0]:.4g},{v[1]:.4g})" if isinstance(v, (list, tuple)) else "--"
    )

    cols = ["x0","SD_iters","TR_iters","SD_fx_final","TR_fx_final",
            "SD_grad_norm_final","TR_grad_norm_final","SD_x_final","TR_x_final"]

    header = " & ".join([
        "$x_0$","SD iters","TR iters","SD $f(x_k)$","TR $f(x_k)$",
        "SD $\\|\\nabla f\\|$","TR $\\|\\nabla f\\|$","SD $x_k$","TR $x_k$"
    ]) + " \\\\"

    lines = ["\\begin{tabular}{lrrrrrrrr}",
             "\\toprule", header, "\\midrule"]

    for _, row in tbl[cols].iterrows():
        x0s = f"({row['x0'][0]:.2g},{row['x0'][1]:.2g})"
        line = " & ".join([
            x0s,
            str(row['SD_iters']), str(row['TR_iters']),
            row['SD_fx_final'], row['TR_fx_final'],
            row['SD_grad_norm_final'], row['TR_grad_norm_final'],
            row['SD_x_final'], row['TR_x_final']
        ]) + " \\\\"
        lines.append(line)

    lines += ["\\bottomrule","\\end{tabular}"]

    with open(tex_path, "w", encoding="utf-8") as ft:
        ft.write("\n".join(lines))

# ---------- Experimentos ----------
def run_all():
    # >= 10 puntos iniciales, todos en una región razonable
    inits = [
        (2.0, 2.0),
        (-2.0, 1.0),
        (0.5, -1.5),
        (3.0, 0.0),
        (-3.0, 0.0),
        (0.0, 3.0),
        (0.0, -3.0),
        (1.5, -2.5),
        (-1.5, 2.5),
        (0.1, 0.1),
    ]

    results = []
    traj_sd_map, traj_tr_map = {}, {}
    hist_sd_map, hist_tr_map = {}, {}

    for x0 in inits:
        traj_sd, hist_sd = steepest_descent(x0, alpha0=1.0)
        traj_tr, hist_tr = trust_region(x0, Delta0=1.0)

        traj_sd_map[x0], traj_tr_map[x0] = traj_sd, traj_tr
        hist_sd_map[x0], hist_tr_map[x0] = hist_sd, hist_tr

        sd_last = hist_sd.iloc[-1] if len(hist_sd) else {'fx': np.nan, 'ng': np.nan}
        tr_last = hist_tr.iloc[-1] if len(hist_tr) else {'fx': np.nan, 'ng': np.nan}

        results.append({
            'x0': x0,
            'SD_iters': len(hist_sd),
            'TR_iters': len(hist_tr),
            'SD_fx_final': float(sd_last['fx']) if len(hist_sd) else np.nan,
            'TR_fx_final': float(tr_last['fx']) if len(hist_tr) else np.nan,
            'SD_grad_norm_final': float(sd_last['ng']) if len(hist_sd) else np.nan,
            'TR_grad_norm_final': float(tr_last['ng']) if len(hist_tr) else np.nan,
            'SD_x_final': traj_sd[-1].tolist() if len(traj_sd) else [np.nan, np.nan],
            'TR_x_final': traj_tr[-1].tolist() if len(traj_tr) else [np.nan, np.nan],
        })

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv("comparison_results.csv", index=False)

    # Figuras
    plot_surface_and_contours()
    plot_trajectories_over_contour(
        traj_sd_map,
        "Trayectorias - Máximo descenso (Armijo)",
        "fig_traj_sd.png"
    )
    plot_trajectories_over_contour(
        traj_tr_map,
        "Trayectorias - Región de confianza (Cauchy)",
        "fig_traj_tr.png"
    )
    plot_convergence(hist_sd_map, "Máximo descenso", "fig_conv_sd.png")
    plot_convergence(hist_tr_map, "Región de confianza", "fig_conv_tr.png")

    # Tabla LaTeX
    save_table_tex(comparison_df, "comparison_table.tex")

    return comparison_df

if __name__ == "__main__":
    df = run_all()
    print(df)
